# Set Coding Environment

## Library installations

In [ ]:
# !pip install geemap geopandas

## Module Imports

In [1]:
# import geopandas as gpd
from pprint import pprint
import ee
from datetime import datetime, timedelta
import pandas as pd
from dateutil.relativedelta import relativedelta

## Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/LULC_Experiments_Chahat_Ananjan_Saketh

/content/drive/MyDrive/LULC_Experiments_Chahat_Ananjan_Saketh


## Authenticate to Google Earth Engine

In [4]:
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=gGoGpevudXOZRpNUswgcif76E_6dxobpsF-lrot5U4g&tc=9YL9Ieq1yp4aOq8ljl_rgwr1U8SObbbxNirJvPVBhlI&cc=GQoS91-omMkcquLJ-u3NJOx6uCQrdE1cmIOK3o6o_dI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXn4erNLJh4W5dDTDB-CI7344fDmiSCyFCW4Tc-JosaUhdSSPZ8sc4g

Successfully saved authorization token.


In [5]:
ee.Initialize()

# Function Definitions

## Defining constants

In [6]:
chastainBandNames = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']

# Regression model parameters from Table-4. MSI TOA reflectance as a function of OLI TOA reflectance.
msiOLISlopes = [1.0946,1.0043,1.0524,0.8954,1.0049,1.0002]
msiOLIIntercepts = [-0.0107,0.0026,-0.0015,0.0033,0.0065,0.0046]

# Regression model parameters from Table-5. MSI TOA reflectance as a function of ETM+ TOA reflectance.
msiETMSlopes = [1.10601,0.99091,1.05681,1.0045,1.03611,1.04011]
msiETMIntercepts = [-0.0139,0.00411,-0.0024,-0.0076,0.00411,0.00861]

# Regression model parameters from Table-6. OLI TOA reflectance as a function of ETM+ TOA reflectance.
oliETMSlopes =[1.03501,1.00921,1.01991,1.14061,1.04351,1.05271];
oliETMIntercepts = [-0.0055,-0.0008,-0.0021,-0.0163,-0.0045,0.00261]

# Construct dictionary to handle all pairwise combos
chastainCoeffDict = { 'MSI_OLI':[msiOLISlopes,msiOLIIntercepts,1], # check what the third item corresponds to
                      'MSI_ETM':[msiETMSlopes,msiETMIntercepts,1],
                      'OLI_ETM':[oliETMSlopes,oliETMIntercepts,1],
                      'OLI_MSI':[msiOLISlopes,msiOLIIntercepts,0],
                      'ETM_MSI':[msiETMSlopes,msiETMIntercepts,0],
                      'ETM_OLI':[oliETMSlopes,oliETMIntercepts,0]
                    }

## Function defnitions

In [7]:
'''
Function to mask cloudy pixels in Landsat-7
'''
def maskL7cloud(image):
  qa = image.select('BQA')
  mask = qa.bitwiseAnd(1 << 4).eq(0)
  return image.updateMask(mask).select(['B1', 'B2', 'B3' , 'B4' , 'B5' , 'B7']).rename('BLUE', 'GREEN', 'RED' , 'NIR' , 'SWIR1' , 'SWIR2')


'''
Function to mask cloudy pixels in Landsat-8
'''
def maskL8cloud(image):
  qa = image.select('BQA')
  mask = qa.bitwiseAnd(1 << 4).eq(0)
  return image.updateMask(mask).select(['B2', 'B3', 'B4' , 'B5' , 'B6' , 'B7']).rename('BLUE', 'GREEN', 'RED' , 'NIR' , 'SWIR1' , 'SWIR2')


'''
Function to mask clouds using the quality band of Semtinel-2 TOA
'''
def maskS2cloud(image):
  qa = image.select('QA60')
  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));
  return image.updateMask(mask).select(['B2', 'B3', 'B4', 'B8',  'B11', 'B12']).rename(['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])


'''
Get Landsat and Sentinel image collections
'''
def Get_L7_L8_S2_ImageCollections(inputStartDate, inputEndDate, roi_boundary):
  # ------ Landsat 7 TOA
  L7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_8DAY_TOA') \
          .filterDate(inputStartDate, inputEndDate) \
          .filterBounds(roi_boundary) \
          .map(maskL7cloud)
  # print('\n Original Landsat 7 TOA dataset: \n',L7.limit(1).getInfo())
  # print('Number of images in Landsat 7 TOA dataset: \t',L7.size().getInfo())

  # ------ Landsat 8 TOA
  L8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_TOA') \
          .filterDate(inputStartDate, inputEndDate) \
          .filterBounds(roi_boundary) \
          .map(maskL8cloud)
  # print('\n Original Landsat 8 TOA dataset: \n', L8.limit(1).getInfo())
  # print('Number of images in Landsat 8 TOA dataset: \t',L8.size().getInfo())

  # ------ Sentinel-2 TOA
  S2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
          .filterDate(inputStartDate, inputEndDate) \
          .filterBounds(roi_boundary)  \
          .map(maskS2cloud)
  # print('\n Original Sentinel-2 TOA dataset: \n',S2.limit(1).getInfo())
  # print('Number of images in Sentinel 2 TOA dataset: \t',S2.size().getInfo())

  return L7, L8, S2


'''
Function to apply model in one direction
'''
def dir0Regression(img,slopes,intercepts):
  return img.select(chastainBandNames).multiply(slopes).add(intercepts)


'''
Applying the model in the opposite direction
'''
def dir1Regression(img,slopes,intercepts):
  return img.select(chastainBandNames).subtract(intercepts).divide(slopes)


'''
Function to correct one sensor to another
'''
def harmonizationChastain(img, fromSensor,toSensor):
  # Get the model for the given from and to sensor
  comboKey = fromSensor.upper() + '_' + toSensor.upper()
  coeffList = chastainCoeffDict[comboKey]
  slopes = coeffList[0]
  intercepts = coeffList[1]
  direction = ee.Number(coeffList[2])

  # Apply the model in the respective direction
  out = ee.Algorithms.If(direction.eq(0),dir0Regression(img,slopes,intercepts),dir1Regression(img,slopes,intercepts))
  return ee.Image(out).copyProperties(img).copyProperties(img,['system:time_start'])


'''
Calibrate Landsat-8 (OLI) and Sentinel-2 (MSI) to Landsat-7 (ETM+)
'''
def Harmonize_L7_L8_S2(L7, L8, S2):
  # harmonization
  harmonized_L8 = L8.map( lambda img: harmonizationChastain(img, 'OLI','ETM') )
  harmonized_S2 = S2.map( lambda img: harmonizationChastain(img, 'MSI','ETM') )

  # Merge harmonized landsat-8 and sentinel-2 to landsat-7 image collection
  harmonized_LandsatSentinel_ic = ee.ImageCollection(L7.merge(harmonized_L8).merge(harmonized_S2))
  # print(harmonized_LandsatSentinel_ic.size().getInfo())
  return harmonized_LandsatSentinel_ic


'''
Add NDVI band to harmonized image collection
'''
def addNDVI(image):
  return image.addBands(image.normalizedDifference(['NIR', 'RED']).rename('NDVI')).float()


'''
Function definitions to get NDVI values at each 16-day composites
'''
def Get_NDVI_image_datewise(harmonized_LS_ic):
  def get_NDVI_datewise(date):
    return harmonized_LS_ic.select(['NDVI']) \
                            .filterDate(ee.Date(date), ee.Date(date).advance(16, 'day')) \
                            .median() \
                            .set('system:time_start',ee.Date(date).millis())
  return get_NDVI_datewise

def Get_LS_16Day_NDVI_TimeSeries(inputStartDate, inputEndDate, harmonized_LS_ic):
  startDate = datetime.strptime(inputStartDate,"%Y-%m-%d")
  endDate = datetime.strptime(inputEndDate,"%Y-%m-%d")

  date_list = pd.date_range(start=startDate, end=endDate, freq='16D').tolist()
  date_list = ee.List( [datetime.strftime(curr_date,"%Y-%m-%d") for curr_date in date_list] )

  LSC =  ee.ImageCollection.fromImages(date_list.map(Get_NDVI_image_datewise(harmonized_LS_ic)))

  return LSC


'''
Pair available LSC and modis values for each time stamp.
'''
def pairLSModis(lsRenameBands):
  def pair(feature):
    date = ee.Date( feature.get('system:time_start') )
    startDateT = date.advance(-8,'day')
    endDateT = date.advance(8,'day')

    # ------ MODIS VI ( We can add EVI to the band list later )
    modis = ee.ImageCollection('MODIS/061/MOD13Q1') \
              .filterDate(startDateT, endDateT) \
              .select(['NDVI','SummaryQA']) \
              .filterBounds(roi_boundary) \
              .median() \
              .rename(['NDVI_modis', 'SummaryQA_modis'])

    return feature.rename(lsRenameBands).addBands(modis)
  return pair


'''
Function to get Pearson Correlation Coffecient to perform GapFilling
'''
def get_Pearson_Correlation_Coefficients(LSC_modis_paired_ic, roi_boundary, bandList):
  corr = LSC_modis_paired_ic.filterBounds(roi_boundary) \
                            .select(bandList).toArray() \
                            .arrayReduce( reducer = ee.Reducer.pearsonsCorrelation(), axes=[0], fieldAxis=1 ) \
                            .arrayProject([1]).arrayFlatten([['c', 'p']])
  return corr


'''Use print(...) to write to this console.
Fill gaps in LSC timeseries using modis data
'''
def gapfillLSM(LSC_modis_regression_model, LSC_bandName, modis_bandName):
  def peformGapfilling(image):
    offset = LSC_modis_regression_model.select('offset')
    scale = LSC_modis_regression_model.select('scale')
    nodata = -1

    lsc_image = image.select(LSC_bandName)
    modisfit = image.select(modis_bandName).multiply(scale).add(offset)

    mask = lsc_image.mask()#update mask needs an input (no default input from the API document)
    gapfill = lsc_image.unmask(nodata)
    gapfill = gapfill.where(mask.Not(), modisfit)

    '''
    in SummaryQA,
    0: Good data, use with confidence
    1: Marginal data, useful but look at detailed QA for more information
    2: Pixel covered with snow/ice
    3: Pixel is cloudy
    '''
    qc_m = image.select('SummaryQA_modis').unmask(3)  # missing value is grouped as cloud
    w_m  = modisfit.mask().rename('w_m').where(qc_m.eq(0), 0.8)  # default is 0.8
    w_m = w_m.where(qc_m.eq(1), 0.5)   # Marginal
    w_m = w_m.where(qc_m.gte(2), 0.2) # snow/ice or cloudy

    # make sure these modis values are read where there is missing data from LandSat, Sentinel
    w_l = gapfill.mask() # default is 1
    w_l = w_l.where(mask.Not(), w_m)

    return gapfill.addBands(w_l).rename(['gapfilled_'+LSC_bandName,'SummaryQA']) #have NDVI from modis and a summary of clarity for each

  return peformGapfilling


'''
Function to combine LSC with Modis data
'''
def Combine_LS_Modis(LSC):
  lsRenameBands = ee.Image(LSC.first()).bandNames().map( lambda band: ee.String(band).cat('_lsc') )
  LSC_modis_paired_ic = LSC.map( pairLSModis(lsRenameBands) )

  # Output contains scale, offset i.e. two bands
  LSC_modis_regression_model_NDVI = LSC_modis_paired_ic.select(['NDVI_modis', 'NDVI_lsc']) \
                                                        .reduce(ee.Reducer.linearFit())

  corr_NDVI = get_Pearson_Correlation_Coefficients(LSC_modis_paired_ic, roi_boundary, ['NDVI_modis', 'NDVI_lsc'])
  LSMC_NDVI = LSC_modis_paired_ic.map(gapfillLSM(LSC_modis_regression_model_NDVI, 'NDVI_lsc', 'NDVI_modis'))

  return LSMC_NDVI


'''
Mask out low quality pixels
'''
def mask_low_QA(lsmc_image):
  low_qa = lsmc_image.select('SummaryQA').neq(0.2)
  return lsmc_image.updateMask(low_qa).copyProperties(lsmc_image, ['system:time_start'])


'''
Add image timestamp to each image in time series
'''
def add_timestamp(image):
  timeImage = image.metadata('system:time_start').rename('timestamp')
  timeImageMasked = timeImage.updateMask(image.mask().select(0))
  return image.addBands(timeImageMasked)


'''
Perform linear interpolation on missing values
'''
def performInterpolation(image):
  image = ee.Image(image)
  beforeImages = ee.List(image.get('before'))
  beforeMosaic = ee.ImageCollection.fromImages(beforeImages).mosaic()
  afterImages = ee.List(image.get('after'))
  afterMosaic = ee.ImageCollection.fromImages(afterImages).mosaic()

  # Interpolation formula
  # y = y1 + (y2-y1)*((t – t1) / (t2 – t1))
  # y = interpolated image
  # y1 = before image
  # y2 = after image
  # t = interpolation timestamp
  # t1 = before image timestamp
  # t2 = after image timestamp

  t1 = beforeMosaic.select('timestamp').rename('t1')
  t2 = afterMosaic.select('timestamp').rename('t2')
  t = image.metadata('system:time_start').rename('t')
  timeImage = ee.Image.cat([t1, t2, t])
  timeRatio = timeImage.expression('(t - t1) / (t2 - t1)', {
                  't': timeImage.select('t'),
                  't1': timeImage.select('t1'),
                  't2': timeImage.select('t2'),
              })

  interpolated = beforeMosaic.add((afterMosaic.subtract(beforeMosaic).multiply(timeRatio)))
  result = image.unmask(interpolated)
  fill_value = ee.ImageCollection([beforeMosaic, afterMosaic]).mosaic()
  result = result.unmask(fill_value)

  return result.copyProperties(image, ['system:time_start'])


def interpolate_timeseries(S1_TS):
  lsmc_masked = S1_TS.map(mask_low_QA)
  filtered = lsmc_masked.map(add_timestamp)

  # Time window in which we are willing to look forward and backward for unmasked pixel in time series
  timeWindow = 120

  # Define a maxDifference filter to find all images within the specified days. Convert days to milliseconds.
  millis = ee.Number(timeWindow).multiply(1000*60*60*24)
  # Filter says that pick only those timestamps which lie between the 2 timestamps not more than millis difference apart
  maxDiffFilter = ee.Filter.maxDifference(
                              difference = millis,
                              leftField = 'system:time_start',
                              rightField = 'system:time_start',
                            )

  # Filter to find all images after a given image. Compare the image's timstamp against other images.
  # Images ahead of target image should have higher timestamp.
  lessEqFilter = ee.Filter.lessThanOrEquals(
                            leftField = 'system:time_start',
                            rightField = 'system:time_start'
                          )

  # Similarly define this filter to find all images before a given image
  greaterEqFilter = ee.Filter.greaterThanOrEquals(
                            leftField = 'system:time_start',
                            rightField = 'system:time_start'
                          )

  # Apply first join to find all images that are after the target image but within the timeWindow
  filter1 = ee.Filter.And( maxDiffFilter, lessEqFilter )
  join1 = ee.Join.saveAll(
                  matchesKey = 'after',
                  ordering = 'system:time_start',
                  ascending = False
          )
  join1Result = join1.apply(
                  primary = filtered,
                  secondary = filtered,
                  condition = filter1
                )

  # Apply first join to find all images that are after the target image but within the timeWindow
  filter2 = ee.Filter.And( maxDiffFilter, greaterEqFilter )
  join2 = ee.Join.saveAll(
                  matchesKey = 'before',
                  ordering = 'system:time_start',
                  ascending = True
          )
  join2Result = join2.apply(
                  primary = join1Result,
                  secondary = join1Result,
                  condition = filter2
                )

  interpolated_S1_TS = ee.ImageCollection(join2Result.map(performInterpolation))

  return interpolated_S1_TS


'''
Function Definition to get Padded NDVI LSMC timeseries image for a given ROI
'''
def Get_Padded_NDVI_TS_Image(startDate, endDate, roi_boundary):
  L7, L8, S2 = Get_L7_L8_S2_ImageCollections(startDate, endDate, roi_boundary)

  harmonized_LS_ic = Harmonize_L7_L8_S2(L7, L8, S2)
  harmonized_LS_ic = harmonized_LS_ic.map(addNDVI)
  LSC = Get_LS_16Day_NDVI_TimeSeries(startDate, endDate, harmonized_LS_ic)
  LSMC_NDVI = Combine_LS_Modis(LSC)
  Interpolated_LSMC_NDVI = interpolate_timeseries(LSMC_NDVI)
  final_LSMC_NDVI_TS = Interpolated_LSMC_NDVI.select(['gapfilled_NDVI_lsc']).toBands()
  final_LSMC_NDVI_TS = final_LSMC_NDVI_TS.clip(roi_boundary)

  input_bands = final_LSMC_NDVI_TS.bandNames()
  return final_LSMC_NDVI_TS, input_bands


'''
Function definition to compute euclidean distance to each cluster centroid
features ---> clusters
flattened ---> time series image clipped to target area
input_bands ---> band names for time series image
studyarea ---> geometry of region of interest
'''
# Function to get distances as required from each pixel to each cluster centroid
def Get_Euclidean_Distance(cluster_centroids, roi_timeseries_img, input_bands, roi_boundary):

  def wrapper(curr_centroid):
    temp_img = ee.Image()
    curr_centroid = ee.Feature(curr_centroid).setGeometry(roi_boundary)
    temp_fc = ee.FeatureCollection( [curr_centroid] )
    class_img = temp_fc.select(['class']).reduceToImage(['class'], ee.Reducer.first()).rename(['class'])
    def create_img(band_name):
      return temp_fc.select([band_name]).reduceToImage([band_name], ee.Reducer.first()).rename([band_name])

    temp_img = input_bands.map(create_img)
    empty = ee.Image()
    temp_img = ee.Image( temp_img.iterate( lambda img, prev: ee.Image(prev).addBands(img) , empty))

    temp_img = temp_img.select(temp_img.bandNames().remove('constant'))
    distance = roi_timeseries_img.spectralDistance(temp_img, 'sed')
    confidence = ee.Image(1.0).divide(distance).rename(['confidence'])
    distance = distance.addBands(confidence)
    return distance.addBands(class_img.rename(['class'])).set('class', curr_centroid.get('class'))

  return cluster_centroids.map(wrapper)


'''
Function definition to get final prediction image from distance images
'''
def Get_final_prediction_image(distance_imgs_list):
  # Denominator is an image that is sum of all confidences to each cluster
  sum_of_distances = ee.ImageCollection( distance_imgs_list ).select(['confidence']).sum().unmask()
  distance_imgs_ic = ee.ImageCollection( distance_imgs_list ).select(['distance','class'])

  # array is an image where distance band is an array of distances to each cluster centroid and class band is an array of classes associated with each cluster
  array_img = ee.ImageCollection(distance_imgs_ic).toArray()

  axes = {'image': 0, 'band':1}
  sort = array_img.arraySlice(axes['band'], 0, 1)
  sorted = array_img.arraySort(sort)

  # take the first image only
  values = sorted.arraySlice(axes['image'], 0, 1)
  # convert back to an image
  min = values.arrayProject([axes['band']]).arrayFlatten([['distance', 'class']])
  # extract dissimilarity score
  dissimilarity_score_img = min.select(0)
  # Extract the hard classification
  predicted_class_img = min.select(1)

  # predicted_class_img = predicted_class_img.add(ee.Image(1)).rename(['Predicted_L3_Label'])
  predicted_class_img = predicted_class_img.rename(['Predicted_L3_Label'])

  confidence_image = ee.Image(1.0).divide(dissimilarity_score_img).rename(['confidence_L3'])
  confidence_image = confidence_image.divide(sum_of_distances)

  final_image = predicted_class_img.addBands(confidence_image)
  return final_image


# Original Pan-India Cropland Time Series

### Get user input

In [8]:
# example- projects/ee-indiasat/assets/IndiaSat_test_polygons
# example- projects/ee-indiasat/assets/India_Boundary
# example- projects/ee-indiasat/assets/india_district_boundaries
# example- projects/ee-indiasat/assets/India_state_boundaries
# example- projects/ee-indiasat/assets/mandya_jaltol_boundary
# example- projects/ee-ananjan/assets/Wassan
# example- projects/ee-indiasat/assets/Polygon_Groundtruth/CSEI_corrected_18July
roi_shapefile_path = input("\n Enter the shapefile path containing your Region Of Interest (ROI): \t")

choice = input("\n Do you want to enter name of your ROI to filter within the shapefile?  \n 1. Yes \n 2. No \n")

if choice == '1':
  roi_name = input("\n Enter the name of your ROI : \t")
  filename_prefix = roi_name
else:
  roi_name = ''
  filename_prefix = input('\nEnter the output filename prefix of your prediction maps (areaName): \t')

# Read the shapefile as feature collection
if roi_name == '':
  roi_boundary = ee.FeatureCollection(roi_shapefile_path)
else:
  roi_boundary = ee.FeatureCollection(roi_shapefile_path).filter(ee.Filter.eq('Name',roi_name))

# pprint(roi_boundary.getInfo())


 Enter the shapefile path containing your Region Of Interest (ROI): 	projects/ee-indiasat/assets/india_district_boundaries

 Do you want to enter name of your ROI to filter within the shapefile?  
 1. Yes 
 2. No 
1

 Enter the name of your ROI : 	Baran


In [ ]:
 pprint(roi_boundary.getInfo())

{'columns': {'Name': 'String',
             'altitudeMo': 'String',
             'begin': 'String',
             'descriptio': 'String',
             'drawOrder': 'String',
             'end': 'String',
             'extrude': 'Long',
             'icon': 'String',
             'snippet': 'String',
             'system:index': 'String',
             'tessellate': 'Long',
             'timestamp': 'String',
             'visibility': 'Long'},
 'features': [{'geometry': {'coordinates': [[[72.34020149196171,
                                              24.62299100215152],
                                             [72.34797817972141,
                                              24.621323292220897],
                                             [72.35378390586439,
                                              24.61809940178043],
                                             [72.35309727272453,
                                              24.616248904701333],
                            

In [ ]:
# roi_boundary = ee.FeatureCollection('users/mtpictd/agro_eco_regions').filter(ee.Filter.eq('ae_regcode',13))
# filename_prefix = 'EcoRegion13'

### Perform classification and save the result for all years

In [9]:
cluster_centroids = ee.FeatureCollection('projects/ee-indiasat/assets/L3_LULC_Clusters/Non_Padded_Original_Clusters_WithLabels')
# cluster_centroids = ee.FeatureCollection('projects/ee-indiasat/assets/L3_LULC_Clusters/Non_Padded_Original_18Clusters_23distortion_afterCustomization')

'''
INFERENCE CODE
'''
startDate = '2015-07-01'
endDate = '2022-07-01'

loopStart = startDate
loopEnd = (datetime.strptime(endDate,"%Y-%m-%d")+relativedelta(years=1)).strftime("%Y-%m-%d")

while loopStart != loopEnd:
  currStartDate = datetime.strptime(loopStart,"%Y-%m-%d")
  currEndDate = (currStartDate+relativedelta(years=1)-timedelta(days=1))

  loopStart = (currStartDate+relativedelta(years=1)).strftime("%Y-%m-%d")

  currStartDate = currStartDate.strftime("%Y-%m-%d")
  currEndDate = currEndDate.strftime("%Y-%m-%d")

  print("\n EXECUTING L3 LULC PREDICTION FOR ",currStartDate," TO ",currEndDate,"\n")

  curr_filename = filename_prefix + '_' + currStartDate + "_" + currEndDate

  final_LSMC_NDVI_TS, input_bands =  Get_Padded_NDVI_TS_Image(currStartDate, currEndDate, roi_boundary)
  distance_imgs_list = Get_Euclidean_Distance(cluster_centroids, final_LSMC_NDVI_TS, input_bands, roi_boundary)
  # pprint(distance_imgs_list.getInfo())
  final_classified_img = Get_final_prediction_image(distance_imgs_list)
  scale = 30

  final_output_filename = curr_filename+'_Level3_LULCmap_'+str(scale)+'m'
  final_output_assetid = 'projects/ee-indiasat/assets/LULC_Deliverables_WithConfidence/Modified/' + final_output_filename

  # Setup the task
  image_export_task = ee.batch.Export.image.toAsset(
    image = final_classified_img.clip(roi_boundary.geometry()),
    description = final_output_filename,
    assetId = final_output_assetid,
    pyramidingPolicy = {'Predicted_L3_Label': 'mode'},
    scale = scale,
    maxPixels = 1e13,
    crs = 'EPSG:4326'
  )

  image_export_task.start()


 EXECUTING L3 LULC PREDICTION FOR  2015-07-01  TO  2016-06-30 


 EXECUTING L3 LULC PREDICTION FOR  2016-07-01  TO  2017-06-30 


 EXECUTING L3 LULC PREDICTION FOR  2017-07-01  TO  2018-06-30 


 EXECUTING L3 LULC PREDICTION FOR  2018-07-01  TO  2019-06-30 


 EXECUTING L3 LULC PREDICTION FOR  2019-07-01  TO  2020-06-30 


 EXECUTING L3 LULC PREDICTION FOR  2020-07-01  TO  2021-06-30 


 EXECUTING L3 LULC PREDICTION FOR  2021-07-01  TO  2022-06-30 


 EXECUTING L3 LULC PREDICTION FOR  2022-07-01  TO  2023-06-30 

